In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

import numpy as np
import tensorflow as tf
#import tensorflow.compat.v1 as tf

c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [14]:

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()
    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299, input_mean=0, input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.io.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(file_reader, channels = 3, name='png_reader')
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(tf.image.decode_gif(file_reader, name='gif_reader'))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
    else:
        image_reader = tf.image.decode_jpeg(file_reader, channels = 3, name='jpeg_reader')
        #print('jpg')
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0);
    resized = tf.image.resize(dims_expander, [input_height, input_width], method=tf.image.ResizeMethod.BILINEAR)
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.compat.v1.Session()
    result = sess.run(normalized)
    return result

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def evalua_modelo(image,modelFile,num):
    file_name='Prueba/'+image
    model_file = './Model_TF1_13_1/'+modelFile+'.pb'
    label_file = './Model_TF1_13_1/'+modelFile+'_labels.txt'
    
    input_height = 64
    input_width = 64
    input_mean = 0
    input_std = 255
    
    input_layer = 'conv2d_'+str(num)+'_input'
    output_layer = 'dense_'+str(num+1)+'/Sigmoid'
    
    graph = load_graph(model_file)
    t = read_tensor_from_image_file(file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    
    input_operation = graph.get_operation_by_name(input_name);
    output_operation = graph.get_operation_by_name(output_name);

    with tf.compat.v1.Session(graph=graph) as sess:
        start = time.time()
        results = sess.run(output_operation.outputs[0],
                          {input_operation.outputs[0]: t})
        end=time.time()
    results = np.squeeze(results)
    top_k = results.argsort()[-5:][::-1]
    labels = load_labels(label_file)
    print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))
    template = "{} (score={:0.5f})"
    a=0
    for i in top_k:
        print(template.format(labels[i], results[i]))
    return labels[top_k[0]]

if __name__ == "__main__":
    image = "Pantene-Hidratación-Extrema.jpg"
    total=''
    ##primer nivel
    modelo = '7-classifier_1_13_1'
    num = 3
    result = evalua_modelo(image,modelo,num)
    total+=result
    print(total)
    ##segundo nivel
    modelo = result
    num = 1
    result = evalua_modelo(image,modelo,num)
    total+=' '+result
    if total=='Jabon Palmolive' or total =='Shampoo Pantene':
        ##tercer nivel
        modelo = result
        num = 1
        result = evalua_modelo(image,modelo,num)
        total+=' '+result
    print('\nEl resultado es: '+total)
        
    


Evaluation time (1-image): 0.154s

Shampoo (score=0.99975)
Pasta Dental (score=0.00196)
Cafe (score=0.00003)
Agua Mineral (score=0.00002)
Jabon (score=0.00001)
Shampoo

Evaluation time (1-image): 0.123s

Pantene (score=1.00000)
Dove (score=0.00000)
Savital (score=0.00000)
Sedal (score=0.00000)
Pert Plus (score=0.00000)

Evaluation time (1-image): 0.146s

Hidratacion Extrema (score=1.00000)
Otro (score=0.00000)
Restauracion (score=0.00000)
Fuerza y Reconstruccion (score=0.00000)

El resultado es: Shampoo Pantene Hidratacion Extrema
